# Case Study 1 : Collecting Data from Twitter

# Problem 1: Sampling Twitter Data with Streaming API about a Certain Topic

* Select and include a case topic that you are interested in, for example, "WPI" or "Lady Gaga" or COVID-19.
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. It would be recommended that the number of tweets should be larger than 1000, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [2]:
import twitter

def oauth_login():
    # XXX: Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
    # for more information on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'OEBQxusAmLaJIP2o3WQTCNxBA'
    CONSUMER_SECRET ='rA29WDveS9w50jF9Ke0edA8xuS2oTcehi2Bq6J11cHPwR4RpQE'
    OAUTH_TOKEN = '1492269919924195329-KdGOMwMHOJZFKTRvm3ha0zysLcg9ev'
    OAUTH_TOKEN_SECRET = 'LQJTZ8HNc5qWleTxhUZYfAy37VJmKH75L9h9OFI2cCRZ0'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

# Sample usage
twitter_api = oauth_login()    

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)
import flask
print(flask.__version__)

1.1.2


In [3]:
import json
import time

def twitter_search(twitter_api, q, max_results=1000, **kw):

    # See https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets
    # and https://developer.twitter.com/en/docs/tweets/search/guides/standard-operators
    # for details on advanced search criteria that may be useful for 
    # keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://developer.twitter.com/en/docs/basics/rate-limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

# Sample usage

twitter_api = oauth_login()

q = 'Beijing2022'
results = twitter_search(twitter_api, q, max_results=1000)
        
# Show one sample search result by slicing the list...
data = json.dumps(results, indent=1)

file = open('extracted_tweets_beijing_2022_final.txt', 'w')
file.write(data)
file.close()

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [4]:
from collections import Counter
from prettytable import PrettyTable
import nltk

tweet_texts = [ tweet['text'] 
                 for tweet in results ]
words = [ word 
          for tweet in tweet_texts 
              for word in tweet.split()
                 if word not in ['RT', '&amp;', '|'] # filter out RT and ampersand
        ]

# Use the natural language toolkit to eliminate stop words

# nltk.download('stopwords') # download stop words if you do not have it
stop_words = nltk.corpus.stopwords.words('english')
non_stop_words = [w for w in words if w.lower() not in stop_words]

# frequency of words
count = Counter(non_stop_words).most_common()

# table of the top 30 words with their counts
pretty_table = PrettyTable(field_names=['Word', 'Count']) 
[ pretty_table.add_row(w) for w in count[:30] ]
pretty_table.align['Word'] = 'l'
pretty_table.align['Count'] = 'r'
print(pretty_table)

+---------------------+-------+
| Word                | Count |
+---------------------+-------+
| #Beijing2022        |   539 |
| #羽生結弦           |   181 |
| #ClosingCeremony    |   145 |
| de                  |   139 |
| @Olympics:          |   134 |
| #フィギュアスケート |   122 |
| 😍                  |   102 |
| 🤩                  |   102 |
| #北京2022           |   101 |
| #EXO                |   100 |
| #엑소               |   100 |
| wait                |    97 |
| #YuzuruHanyu        |    96 |
| Can't               |    96 |
| tonight's           |    96 |
| @weareoneEXO        |    95 |
| rocked              |    95 |
| PyeongChang2018     |    95 |
| 🎶🎤                |    95 |
| ceremony!           |    95 |
| #Grow…              |    95 |
| Olympic             |    76 |
| 選手                |    55 |
| #TeamGB             |    51 |
| エキシビション      |    50 |
| #北京五輪           |    48 |
| #北京オリンピック   |    46 |
| medal               |    46 |
| #Olympics           |    45 |
| la              

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [5]:
# Create a list of all tweets with a retweeted_status key, and index the originator of that tweet and the text.
retweets = [
            (tweet['retweet_count'],
            tweet['text'])
            
            #Ensure that a retweet exists
            for tweet in results                      
            ]

rtdict = dict(list(zip(tweet_texts, retweets)))
def sortRTdict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux
sortedrtdict = sortRTdict(rtdict)
rts = []
tweets = []

for duo in sortedrtdict[:10]:
    tweets.append(duo[1])
    rts.append(duo[0])

pretty_table = PrettyTable(field_names = ['Retweets','Text'])

# Sort tweets by descending number of retweets and display the top 10 results in a table.
[pretty_table.add_row(row) for row in rts]
pretty_table.max_width['Text'] = 200
pretty_table.align = 'l'
print(pretty_table)

+----------+---------------------------------------------------------------------------------------------+
| Retweets | Text                                                                                        |
+----------+---------------------------------------------------------------------------------------------+
| 18641    | RT @Japan_Olympic: ／                                                                       |
|          | 決勝は明日！ #カーリング 女子🥌                                                             |
|          | 選手たちからメッセージが届きました✨🙆‍♀️                                                     |
|          | ＼                                                                                          |
|          |                                                                                             |
|          | #がんばれニッポン 🇯🇵💪                                                                      |
|          |                                                                   

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags and top 10 user mentions that are the most popular in your collection of tweets.

In [6]:
# Extract the screen names which appear among the collection of tweets
screen_names = [user_mention['screen_name']
               for tweet in results
                   for user_mention in tweet['entities']['user_mentions']]

# Extract the hashtags which appear among the collection of tweets
hashtags = [ hashtag['text']
           for tweet in results
               for hashtag in tweet['entities']['hashtags']]

# Simultaneously determine the frequency of screen names/hashtags, and display the top 10 most common in a table.
for label, data in (('Screen Name',screen_names),
                   ('Hashtag',hashtags)):
    pretty_table = PrettyTable(field_names =[label,'Count'])
    counter = Counter(data)
    [ pretty_table.add_row(entity) for entity in counter.most_common()[:10]]
    pretty_table.align[label] ='l'
    pretty_table.align['Count'] = 'r'
    print(pretty_table)

+----------------+-------+
| Screen Name    | Count |
+----------------+-------+
| Olympics       |   154 |
| weareoneEXO    |    95 |
| Beijing2022    |    82 |
| TeamGB         |    58 |
| gorinjp        |    34 |
| jiji_shashinbu |    31 |
| mainichiphoto  |    27 |
| kyodo_photo    |    25 |
| nhk_sports     |    24 |
| Japan_Olympic  |    24 |
+----------------+-------+
+--------------------+-------+
| Hashtag            | Count |
+--------------------+-------+
| Beijing2022        |   602 |
| 羽生結弦           |   188 |
| フィギュアスケート |   159 |
| ClosingCeremony    |   145 |
| 北京2022           |   101 |
| YuzuruHanyu        |   101 |
| EXO                |   100 |
| 엑소               |   100 |
| 北京五輪           |    94 |
| 北京オリンピック   |    93 |
+--------------------+-------+


# Problem 3: Getting "All" Friends and "All" Followers of a Popular User in Twitter

* Choose a popular twitter user who has many followers in the collected tweets of your case topic.
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers, and their screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers, and their screen names in a table.

In [5]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary







* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [6]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary








# Problem 4: Domain Question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a question, which Twitter data could help answer from your collected tweets, based upon your case topic of your chosen public organization, private company, social community, etc., in a domain.
* How could Twitter data help that company/organization/community spend its resources to answer the above question?

In [7]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary



